In [ ]:
import os
import sys
from typing import Dict, Any

import numpy as np


project_root = os.path.abspath(os.path.join(os.getcwd(), '../../train', '..'))
app_root = os.path.abspath(os.path.join(project_root, '../../app', '..'))

if project_root not in sys.path:
    sys.path.append(project_root)
    sys.path.append(app_root)

In [ ]:
from datasets_preprocessing import load_json_data, make_pipeline
import pandas as pd

math_pipeline = make_pipeline('math')

X_json_raw = load_json_data('datasets/math')
math_pipeline.fit_transform(X_json_raw)

math_df = pd.read_csv(os.path.join('csv_question_files', 'math.csv'))
math_df.head(10)

In [ ]:
bio_pipeline = make_pipeline('bio')

X_json_raw = load_json_data('datasets/bio')
bio_pipeline.fit_transform(X_json_raw)

bio_df = pd.read_csv(os.path.join('csv_question_files', 'bio.csv'))
bio_df.head(10)

In [ ]:
code_pipeline = make_pipeline('code')

X_json_raw = load_json_data('datasets/code')
code_pipeline.fit_transform(X_json_raw)

code_df = pd.read_csv(os.path.join('csv_question_files', 'code.csv'))
code_df.head(10)

In [ ]:
n_samples = 6000
half_samples = n_samples // 2

full_df = pd.concat(
	[
		math_df.sample(n=n_samples, random_state=42),
		bio_df.sample(n=half_samples , random_state=42),
		code_df.sample(n=half_samples, random_state=42)
	],
    ignore_index=True,
	axis=0
)

full_df

# Math Model

## SVM + TF-IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from train.reporting.sentence_transformer_sklearn_adapter import SentenceTransformerSklearnAdapter
from sklearn.pipeline import Pipeline
from train.reporting.model_interface import ModelInterface
from sklearn.svm import SVC

In [ ]:
class TextSVMWrapper(ModelInterface):
    def get_params(self) -> Dict[str, Any]:
	    return self.model.get_params()

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
	    return self.model.predict_proba(X)

    def get_vectorizer(self):
	    return self.model.steps[0][1]

    def __init__(self, C=1.0):
        # Pipeline: Najpierw zamiana tekstu na liczby (TF-IDF), potem klasyfikator SVM
        self.model = Pipeline([
            ('tfidf', SentenceTransformerSklearnAdapter()),
            ('svm', SVC(C=C, random_state=42, probability=True, kernel='linear'))
        ])
        self.C = C
        self.is_fitted = False

    def fit(self, X, y, X_val=None, y_val=None):
        # SVM w sklearn nie wspiera śledzenia historii loss per epoka w prosty sposób,
        # więc po prostu trenujemy model.
        self.model.fit(X, y)
        self.is_fitted = True

    def predict(self, X):
        return self.model.predict(X)

    def get_loss_history(self):
        # SVM ze sklearn nie udostępnia historii funkcji straty w czasie treningu.
        # Zwracamy pusty słownik, aby Reporter wiedział, że ma pominąć wykres.
        return {}

    def get_new_instance(self):
        # Zwraca nową, czystą instancję (potrzebne do Cross Validation w Reporterze)
        return TextSVMWrapper(C=self.C)

    def get_feature_importance(self):
        """
        Dla tekstu 'ważność cech' to słowa, które najsilniej wskazują na daną kategorię.
        """
        if not self.is_fitted:
            return {}

        try:
            # Pobieramy słowa z wektoryzatora
            feature_names = self.model.named_steps['tfidf'].get_feature_names_out()
            # Pobieramy wagi z SVM
            coefs = self.model.named_steps['svm'].coef_.copy()

            # Uwaga: SVM binarny ma 1 wymiar wag, wieloklasowy ma (n_klas, n_cech).
            # Dla uproszczenia bierzemy średnią siłę wpływu słowa (wartość bezwzględna) dla wszystkich klas.
            # To pokaże słowa, które są ogólnie najbardziej "decydujące".
            avg_coefs = np.mean(np.abs(coefs), axis=0)
            avg_coefs = np.ravel(avg_coefs)

            # Tworzymy słownik {słowo: waga}
            importance_dict = dict(zip(feature_names, avg_coefs))
            return importance_dict
        except Exception as e:
            print(f"Nie udało się pobrać ważności cech: {e}")
            return {}

In [ ]:
from train.reporting.model_reporter import ModelReporter

report_df = full_df

reporter = ModelReporter(TextSVMWrapper(C=1.0), report_df['question'],
                         report_df['math'])


In [ ]:
reporter.generate_report()

In [ ]:
reporter.plot_tsne()

In [ ]:
reporter.plot_wordclouds()

In [ ]:
reporter.plot_confidence_distribution()